In [1]:
import yt_dlp
import os
import shutil

def download_youtube_mp3(video_id: str = 'gqXEEydkbh4'):
    # FFmpeg 설치 여부 확인
    if shutil.which("ffmpeg") is None:
        print("오류: FFmpeg이 설치되어 있지 않습니다. 설치 후 다시 시도하세요.")
        return None

    url = f"https://www.youtube.com/watch?v={video_id}"
    output_dir = "downloads"
    os.makedirs(output_dir, exist_ok=True)  # 다운로드 폴더 생성

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f"{output_dir}/%(id)s.%(ext)s",
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            file_name = f"{info['id']}.mp3"
            file_path = os.path.join(output_dir, file_name)

        print(f"다운로드 완료! 파일 경로: {file_path}")
        return file_path
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

# 테스트 실행 gqXEEydkbh4
audio_path = download_youtube_mp3('GmRsuVTMqOw')
print(audio_path)

[youtube] Extracting URL: https://www.youtube.com/watch?v=GmRsuVTMqOw
[youtube] GmRsuVTMqOw: Downloading webpage
[youtube] GmRsuVTMqOw: Downloading tv client config
[youtube] GmRsuVTMqOw: Downloading player f6e09c70
[youtube] GmRsuVTMqOw: Downloading tv player API JSON
[youtube] GmRsuVTMqOw: Downloading ios player API JSON
[youtube] GmRsuVTMqOw: Downloading m3u8 information
[info] GmRsuVTMqOw: Downloading 1 format(s): 251
[download] Destination: downloads\GmRsuVTMqOw.webm
[download] 100% of    7.69MiB in 00:00:00 at 9.06MiB/s   
[ExtractAudio] Destination: downloads\GmRsuVTMqOw.mp3
Deleting original file downloads\GmRsuVTMqOw.webm (pass -k to keep)
다운로드 완료! 파일 경로: downloads\GmRsuVTMqOw.mp3
downloads\GmRsuVTMqOw.mp3


In [2]:
import ffmpeg
import os

def trim_audio_file(file_path:str, max_size:int=24*1024*1024):
    file_size = os.path.getsize(file_path)  # 바이트 단위
    output_path = file_path
    if file_size >= max_size:
        # ffmpeg로 오디오 비트레이트 가져오기
        probe = ffmpeg.probe(file_path)
        bitrate = int(probe['format']['bit_rate'])  # 비트레이트 (bps)

        # 최대 24MB로 자를 수 있는 재생 시간 계산
        max_size = 24*1024*1024 # 25MB 보다 작게
        max_duration = max_size * 8 / bitrate  # 초 단위로 변환

        # 출력 파일 경로 설정
        output_path = file_path.replace(".mp3", "_trimmed.mp3")

        # ffmpeg로 파일 자르기
        ffmpeg.input(file_path, ss=0, t=max_duration).output(output_path, codec="copy").run(overwrite_output=True)
    return output_path

trimmed_audio_path = trim_audio_file(audio_path)

In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    api_key = api_key
)

In [4]:
def transcribe_audio(audio_path):
    audio_file= open(audio_path, "rb")
    response = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        response_format="text"
    )
    return response

# 텍스트 변환 실행
transcribed_text = transcribe_audio(trimmed_audio_path)
print("변환된 텍스트:\n", transcribed_text)

변환된 텍스트:
 There is a rollercoaster happening with regard to the tariffs, those 25 percent tariffs that the president slapped on Canada and Mexico on Monday. Well, those are apparently not happening, at least when it comes to Canada and, excuse me, to Mexico and not Canada. He is reversing some of them, at least maybe. Frankly, the reason why it's hard for us to describe is because nobody knows what exactly is happening. The president's commerce secretary said this morning that he was likely to postpone them for at least a month. But then minutes ago, President Trump said that they are going to delay those tariffs for Mexico, but not Canada. Washington, Wall Street and Main Street have a lot riding on this. Everybody is extremely confused. Maybe Jeff Zeleny is not one of those people. Jeff, you're standing outside the White House. You're on the North Lawn. Can you give us any sense of what all this means? Well, Dana, if there's any question of why the market has been so confused and rat

In [5]:
# 변역
def translate(text, lang = "한국어"):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            { "role": "system", "content": "주어진 텍스트를 {lang}로 번역해주세요." },
            { "role": "user", "content": text }
        ],
    )
    print(f"텍스트를 {lang}으로 번역하였습니다.")
    return response.choices[0].message.content

translated_text = translate(transcribed_text)

텍스트를 한국어으로 번역하였습니다.


In [6]:
# 음성으로 전환(TTS) : 4,096 바이트 단위로 나눠서 요청하고 병합
# 4096 바이트 단위로 텍스트 나누기
def split_text_by_bytes(text, max_bytes=4096):
    chunks = []
    encoded_text = text.encode("utf-8")
    start = 0
    while start < len(encoded_text):
        chunk = encoded_text[start:start+max_bytes]
        chunks.append(chunk.decode("utf-8", errors="ignore"))
        start += max_bytes
    return chunks

In [7]:
# MP3 파일 저장 및 API 호출
def generate_tts(chunks):
    audio_files = []
    for i, chunk in enumerate(chunks):
        response = client.audio.speech.create(
            model="tts-1",
            voice="alloy",  # 선택 가능한 음성 (alloy, echo, fable, onyx, nova, shimmer)
            input=chunk
        )
        audio_filename = f"downloads/part_{i}.mp3"
        with open(audio_filename, "wb") as f:
            f.write(response.content)
        audio_files.append(audio_filename)
    return audio_files

In [9]:
from pydub import AudioSegment

# MP3 파일 병합
def merge_audio(files, output_file):
    # MP3 파일로 저장
    output_filename = output_file.replace(".mp3", "_translated.mp3")
    combined = AudioSegment.empty()
    for file in files:
        sound = AudioSegment.from_mp3(file)
        combined += sound
    combined.export(output_filename, format="mp3")
    print(f"MP3 파일이 병합되었습니다: {output_filename}")

In [10]:
# 실행
chunks = split_text_by_bytes(translated_text)
audio_files = generate_tts(chunks)
merge_audio(audio_files, audio_path)

MP3 파일이 병합되었습니다: downloads\GmRsuVTMqOw_translated.mp3


In [11]:
print(audio_path)

downloads\GmRsuVTMqOw.mp3


In [ ]:
import ffmpeg

def adjust_audio_speed(input_file, output_file, speed_factor):
    """
    입력된 오디오 파일의 속도를 조절하여 새로운 파일로 저장합니다.

    :param input_file: 입력 파일 경로 (예: 'input.mp3')
    :param output_file: 출력 파일 경로 (예: 'output.mp3')
    :param speed_factor: 속도 조절 비율 (1.0은 원본 속도, 1.2는 1.2배 빠름)
    """
    try:
        # FFmpeg 명령어로 속도 조정
        ffmpeg.input(input_file).filter('atempo', speed_factor).output(output_file).run()
        print(f"속도가 조정된 오디오 파일이 '{output_file}'로 저장되었습니다.")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

# 예시 사용
adjust_audio_speed(audio_path, audio_path.replace(".mp3", "_1.2.mp3"), 1.2)  # 1.2배 빠른 속도